# Enhancing Singapore Airlines' Service Through Automated Sentiment Analysis of Customer Reviews



**Motivation**



## Singapore Airlines Customer Reviews Dataset Information

The [Singapore Airlines Customer Reviews Dataset](https://www.kaggle.com/datasets/kanchana1990/singapore-airlines-reviews) aggregates 10,000 anonymized customer reviews, providing a broad perspective on the passenger experience with Singapore Airlines. 

The dimensions are shown below:
- **`published_date`**: Date and time of review publication.
- **`published_platform`**: Platform where the review was posted.
- **`rating`**: Customer satisfaction rating, from 1 (lowest) to 5 (highest).
- **`type`**: Specifies the content as a review.
- **`text`**: Detailed customer feedback.
- **`title`**: Summary of the review.
- **`helpful_votes`**: Number of users finding the review helpful.

## Additional web scraping of online reviews

During our EDA, we noticed two main trends in the distribution of our dataset:
1. Less than 10% of our reviews were published from the years 2022 to 2024, making it hard for us to capture recent trends in sentiment.
2. Most of the reviews were highly positive, which could mean that SIA had mostly positive reviews, nevertheless we wanted to get more information on negative reviews to improve the robustness of our model.

### TripAdvisor

We scraped more data for airline reviews from TripAdvisor, specifically for the years 2022 to 2024. 
(https://www.tripadvisor.com.sg/Airline_Review-d8729151-Reviews-Singapore-Airlines)

The dimensions are shown below:
- **`Year`**: Year of review publication.
- **`Month`**: Month of review publication.
- **`Title`**: Title of review publication.
- **`Review Text`**: Main text content of review publication.
- **`Rating`**: Numerical rating provided by reviewer (Scale: 1 to 5)


### Skytrax

We also scraped from Skytrax, which is another data source for online reviews. 
(https://www.airlinequality.com/airline-reviews/singapore-airlines/?sortby=post_date%3ADesc&pagesize=100)

The dimensions are shown below:
- **`Year`**: Year of review publication.
- **`Month`**: Month of review publication.
- **`Title`**: Title of review publication.
- **`Review Text`**: Main text content of review publication.
- **`Rating`**: Numerical rating provided by reviewer (Scale: 1 to 10)

## Importing Libraries

Please uncomment the code box below to pip install relevant dependencies for this notebook.

In [1]:
!pip3 install -r requirements.txt

In [2]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime 

# Statistical functions
from scipy.stats import zscore

# Text Preprocessing and NLP
import nltk
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.corpus import wordnet

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer


# For generating n-grams
from nltk.util import ngrams
from collections import Counter

## Data Preparation (Loading CSV)

Load the three CSV files into a pandas DataFrame `data`.

In [3]:
data = pd.read_csv('final_df.csv')

In [4]:
data.head()

,year,month,sentiment,processed_full_review
0,2024,3,Neutral,ok use airlin go singapor london heathrow issu...
1,2024,3,Negative,don give money book paid receiv email confirm ...
2,2024,3,Positive,best airlin world best airlin world seat food ...
3,2024,3,Negative,premium economi seat singapor airlin not worth...
4,2024,3,Negative,imposs get promis refund book flight full mont...


In [5]:
data['sentiment'].value_counts()

sentiment
Positive    7913
Negative    2441
Neutral     1164
Name: count, dtype: int64

In [6]:
data['year'].value_counts()

year
2019    5129
2018    2596
2022    1184
2023    1111
2020     888
2024     514
2021      96
Name: count, dtype: int64

## Simple Neural Network

A Simple Neural Network, or fully connected neural network (FCNN), is a basic deep learning model ideal for straightforward classification tasks. It consists mainly of fully connected layers that process flattened data inputs, making it versatile for many types of data, including text.

Below is an explanation of how a simple NN works:

1. Embedding Layer (for Text Data):
	- For text inputs, an embedding layer transforms words into numerical vectors that capture meaning and context.
    
2.	Flattening:
	- The embeddings are flattened into a single long vector, allowing the network to process them as one input.

3.	Dense (Fully Connected) Layers:
	- Dense layers are the core of an FCNN. Each neuron connects to all neurons in the previous layer, learning complex relationships.
	- Activation functions, such as ReLU, are applied here to introduce non-linearity, helping the network capture more intricate patterns.

4.	Output Layer:
	- The final layer outputs class probabilities using a softmax activation (for multi-class classification) or sigmoid (for binary classification).
	- This layer helps the model predict the likelihood of each class for an input.
	
5.	Training:
	- During training, the network adjusts its weights to minimize prediction errors, gradually improving its accuracy through backpropagation.


In [10]:
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Assuming 'data' is your DataFrame with 'processed_full_review' and 'sentiment' columns

# Step 1: Tokenization and Padding
max_words = 10000  # Maximum vocabulary size
max_sequence_length = 300  # Maximum length of sequences

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])

# Pad sequences to ensure uniform length
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode the sentiment labels
onehot_encoder = OneHotEncoder(sparse_output=False)
y = onehot_encoder.fit_transform(data[['sentiment']])

# Step 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Define the Simple Neural Network Model
def create_simple_nn():
    model = Sequential()
    
    # Embedding layer
    model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
    
    # Flatten the embeddings to feed into dense layers
    model.add(Flatten())
    
    # Fully connected layers
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    
    # Output layer for three-class classification using softmax
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # Categorical cross-entropy is used for multi-class classification
    return model

# Step 4: Train the Model
model = create_simple_nn()
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Step 5: Evaluate the Model
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=onehot_encoder.categories_[0], digits=4)

print("\nEvaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)


Epoch 1/10


/Users/mayaung/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.5995 - loss: 0.9576 - val_accuracy: 0.6970 - val_loss: 0.7742
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - accuracy: 0.7046 - loss: 0.7441 - val_accuracy: 0.8355 - val_loss: 0.4297
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - accuracy: 0.8170 - loss: 0.4803 - val_accuracy: 0.8451 - val_loss: 0.4042
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.8520 - loss: 0.3754 - val_accuracy: 0.8446 - val_loss: 0.4085
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.8728 - loss: 0.3040 - val_accuracy: 0.8398 - val_loss: 0.4221
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 9s 59ms/step - accuracy: 0.8929 - loss: 0.2377 - val_accuracy: 0.8485 - val_loss: 0.4917
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.9242 - loss: 0.1763 - val_accuracy: 0.8477 - val_loss: 0.5695
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.9474 - loss: 0.1302 - val_accuracy: 0.84

# Simple Neural Network Accounting for Imbalanced Classes

1. Class Weights Calculation: compute_class_weight calculates class weights based on the training labels, helping to handle the imbalance by giving higher weight to underrepresented classes.

2. Passing class_weight in fit: By adding class_weight=class_weights_dict in model.fit, we inform the model to apply these weights during training, making it more sensitive to the underrepresented classes.

In [11]:
import tensorflow as tf
import numpy as np
import random
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Assuming 'data' is your DataFrame with 'processed_full_review' and 'sentiment' columns

# Step 1: Tokenization and Padding
max_words = 10000  # Maximum vocabulary size
max_sequence_length = 300  # Maximum length of sequences

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])

# Pad sequences to ensure uniform length
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode the sentiment labels
onehot_encoder = OneHotEncoder(sparse_output=False)
y = onehot_encoder.fit_transform(data[['sentiment']])

# Step 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Calculate class weights for imbalanced classes
y_train_integers = np.argmax(y_train, axis=1)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_integers), y=y_train_integers)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Step 3: Define the Simple Neural Network Model
def create_simple_nn():
    model = Sequential()
    
    # Embedding layer
    model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
    
    # Flatten the embeddings to feed into dense layers
    model.add(Flatten())
    
    # Fully connected layers
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    
    # Output layer for three-class classification using softmax
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # Categorical cross-entropy is used for multi-class classification
    return model

# Step 4: Train the Model
model = create_simple_nn()
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), 
                    class_weight=class_weights_dict, verbose=1)

# Step 5: Evaluate the Model
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=onehot_encoder.categories_[0], digits=4)

print("\nEvaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)


Epoch 1/10


/Users/mayaung/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 8s 49ms/step - accuracy: 0.3922 - loss: 1.1366 - val_accuracy: 0.6884 - val_loss: 1.0814
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.6737 - loss: 1.0927 - val_accuracy: 0.7491 - val_loss: 1.0423
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.6489 - loss: 0.9723 - val_accuracy: 0.8203 - val_loss: 0.4918
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - accuracy: 0.7661 - loss: 0.7756 - val_accuracy: 0.7982 - val_loss: 0.5564
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - accuracy: 0.7986 - loss: 0.6559 - val_accuracy: 0.7977 - val_loss: 0.5238
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 0.8177 - loss: 0.5372 - val_accuracy: 0.7934 - val_loss: 0.5140
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.8601 - loss: 0.4304 - val_accuracy: 0.8225 - val_loss: 0.4858
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.8841 - loss: 0.3440 - val_accuracy: 0.819

# Simple Neural Network + Word2Vec

1. Use the pre-trained Word2Vec embeddings you created for initializing an embedding layer.

2. Define a simple neural network that flattens the embeddings and then feeds them into dense layers.

3. Add class_weight to handle the imbalanced classes.



In [12]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from gensim.models import Word2Vec

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Parameters
vocab_size = 5000         # Limit vocabulary to 5000 words
embedding_dim = 128       # Embedding dimensions for each word
max_sequence_length = 300 # Max number of words in each sequence

# Step 1: Tokenize and Pad the Text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])
X_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Labels
sentiment_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = data['sentiment'].map(sentiment_dict).values

# Step 2: Train Word2Vec Model
sentences = [text.split() for text in data['processed_full_review']]
word2vec_model = Word2Vec(sentences, vector_size=embedding_dim, window=5, min_count=1, workers=4, sg=1)

# Create Embedding Matrix from Trained Word2Vec Model
embedding_matrix = np.zeros((vocab_size, embedding_dim))
word_index = tokenizer.word_index

for word, i in word_index.items():
    if i < vocab_size:
        embedding_vector = word2vec_model.wv[word] if word in word2vec_model.wv else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Step 3: Define Simple Neural Network Model
def create_simple_nn():
    model = Sequential()
    
    # Embedding layer with pre-trained Word2Vec embeddings
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], 
                        input_length=max_sequence_length, trainable=True))
    
    # Flatten layer to feed embeddings into dense layers
    model.add(Flatten())
    
    # Fully connected layers
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    
    # Output layer for three-class classification using softmax
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Step 4: Train the Model
model = create_simple_nn()
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, verbose=1, class_weight=class_weights_dict)

# Step 5: Evaluate the Model
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Classification Report
report = classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive'], zero_division=0, digits=4)
print('Performance Metrics:\n', report)


Epoch 1/10


/Users/mayaung/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.4666 - loss: 1.1120 - val_accuracy: 0.7710 - val_loss: 0.8031
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.6094 - loss: 0.9097 - val_accuracy: 0.7705 - val_loss: 0.6167
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.6905 - loss: 0.8298 - val_accuracy: 0.7792 - val_loss: 0.5281
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.6892 - loss: 0.7299 - val_accuracy: 0.8074 - val_loss: 0.4609
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.7520 - loss: 0.6545 - val_accuracy: 0.8058 - val_loss: 0.4760
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.7722 - loss: 0.5862 - val_accuracy: 0.8047 - val_loss: 0.4613
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.8085 - loss: 0.5083 - val_accuracy: 0.8209 - val_loss: 0.4748
Epoch 8/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.8333 - loss: 0.4712 - val_accuracy: 0.806